# Azure AI Agenti s podporou protokolu Model Context Protocol (MCP)

Tento zápisník ukazuje, ako používať Azure AI agentov s nástrojmi protokolu Model Context Protocol (MCP) na vytvorenie inteligentného agenta, ktorý dokáže využívať externé MCP servery na rozšírené schopnosti.


## Inštalácia potrebných balíkov NuGet

Najskôr musíme nainštalovať balík Azure AI Agents Persistent, ktorý poskytuje základnú funkcionalitu pre prácu s Azure AI Agents.


## Výhody autentifikácie bez kľúčov

Tento notebook demonštruje **autentifikáciu bez kľúčov**, ktorá prináša niekoľko výhod:
- ✅ **Žiadne API kľúče na správu** - Používa autentifikáciu založenú na identite Azure
- ✅ **Zvýšená bezpečnosť** - Žiadne tajné údaje uložené v kóde alebo konfigurácii
- ✅ **Automatická rotácia poverení** - Azure spravuje životný cyklus poverení
- ✅ **Prístup založený na rolách** - Používa Azure RBAC na detailné nastavenie oprávnení

`DefaultAzureCredential` automaticky použije najlepší dostupný zdroj poverení:
1. Spravovaná identita (pri spustení v Azure)
2. Poverenia Azure CLI (počas vývoja)
3. Poverenia Visual Studio
4. Premenné prostredia (ak sú nakonfigurované)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

Nainštalujte balík Azure Identity na autentifikáciu so službami Azure pomocou DefaultAzureCredential.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## Importujte potrebné názvy priestorov

Importujte potrebné názvy priestorov pre Azure AI Agents a Azure Identity.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Konfigurácia Azure AI Agent Client (Autentifikácia bez kľúča)

Nastavte konfiguračné premenné a vytvorte PersistentAgentsClient pomocou **autentifikácie bez kľúča**:
- **projectEndpoint**: Koncový bod projektu Azure AI Foundry
- **modelDeploymentName**: Názov nasadeného AI modelu (GPT-4.1 nano)
- **mcpServerUrl**: URL adresa MCP servera (Microsoft Learn API)
- **mcpServerLabel**: Označenie na identifikáciu MCP servera
- **DefaultAzureCredential**: Používa spravovanú identitu, Azure CLI alebo iné zdroje poverení (nie sú potrebné API kľúče)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## Vytvorenie definície nástroja MCP

Vytvorte definíciu nástroja MCP, ktorý sa pripojí k serveru Microsoft Learn MCP. To umožní agentovi prístup k obsahu a dokumentácii Microsoft Learn.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## Vytvorte AI agenta

Vytvorte trvalého AI agenta so špecifikovaným modelom a nástrojmi MCP. Agent je nakonfigurovaný s:
- Modelom GPT-4.1 nano
- Pokynmi na používanie nástrojov MCP na asistenciu
- Prístupom k serveru Microsoft Learn MCP


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## Vytvorenie vlákna a odoslanie správy

Vytvorte konverzačné vlákno a pošlite používateľovi správu s otázkou o rozdiele medzi Azure OpenAI a OpenAI. Týmto sa otestuje schopnosť agenta používať nástroje MCP na poskytovanie presných informácií.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## Konfigurácia zdrojov nástroja MCP (Bez kľúča)

Nastavte zdroje nástroja MCP. Pre skutočne bezkľúčový prístup môžete odstrániť vlastné hlavičky, ak server MCP podporuje autentifikáciu založenú na identite Azure. Príklad nižšie ukazuje, ako pridať hlavičky, ak je to potrebné, ale pre bezkľúčové scenáre nemusia byť vyžadované.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## Spustiť beh agenta

Vytvorte a spustite beh na spracovanie správy používateľa. Agent použije nakonfigurované MCP nástroje a zdroje na vytvorenie odpovede.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## Monitorovanie spustenia a spracovanie schválení nástrojov (bez kľúča)

Monitorujte stav spustenia agenta a spracujte všetky potrebné schválenia nástrojov. Tento cyklus:
1. Čaká na dokončenie spustenia alebo na potrebu akcie
2. Automaticky schvaľuje volania nástrojov MCP, keď je to potrebné
3. Pri autentifikácii bez kľúča nemusia byť hlavičky potrebné, ak server MCP podporuje identitu Azure


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## Zobraziť výsledky konverzácie

Získajte a zobrazte všetky správy v vlákne, pričom zobrazíte otázku používateľa aj odpoveď agenta. Správy sú zobrazené v chronologickom poradí s časovými pečiatkami a indikátormi rolí.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**Upozornenie**:  
Tento dokument bol preložený pomocou služby AI prekladu [Co-op Translator](https://github.com/Azure/co-op-translator). Hoci sa snažíme o presnosť, prosím, berte na vedomie, že automatizované preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho pôvodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre kritické informácie sa odporúča profesionálny ľudský preklad. Nie sme zodpovední za žiadne nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
